In [1]:
#Custom function to encode nucleotides according to Brain format 
def encodeDNASeq(seq, encoding='OneHot'):
    """Encode nucleotides from character to double or OneHot encoding.
    Using OneHot nucleotides are encoded as:
    A->1000; C->0100; G->0010; T->0001; other->0000
    Using Index as: A->1.0; C->2.0; G->3.0; t->4.0; other->0.0
    @param: seq A string containing a sequence of nucleotides 
    @param: encoding_type output encodig: OneHot or Index
    """    

    if encoding=="Index":
        mymap = {'A':1.0, 'C':2.0, 'G':3.0, 'T':4.0, 'N':0.0}

    else:
        mymap ={'A':SparseVector(4, [0], [1]), 
                'C':SparseVector(4, [1], [1]), 
                'G':SparseVector(4, [2], [1]), 
                'T':SparseVector(4, [3], [1]), 
                'N':SparseVector(4,[0],[0])}    
    
    indexed_seq=list()
    #Verificare se si può sostituire con qualcosa di parallelizzabile
    #Mettere quì la selezione della finestra di osservazione
    for n in seq:
       indexed_seq.append(mymap.get(n) if n in mymap else SparseVector(4, [0], [0]))
    return indexed_seq   

#Split each line in single features
#encode each nucleotide using function encodeDNASeq
def load_dna_dataset(file_name,label_value, nrows=0, encoding='OneHot'):
    "Read Input Dataset contained in file_name. Data are labelled with value specified in label_value parameter"
    rdd = sc.textFile(file_name).flatMap(lambda line: [list(line)]).map(lambda s: encodeDNASeq(s,encoding)) 
        
    #Insert Label Column and convert Rdd into Dataframe in order to apply ML Algorithm
    return rdd.toDF().withColumn("label",lit(label_value))

#Print ROC
def roc(predictions_and_labels):
    "Return ROC: Receiver Operating Characteristic curv"
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    my_bin_evaluator = BinaryClassificationEvaluator()
    return  my_bin_evaluator.evaluate(predictions_and_labels.predictions)


def accuracy(predictions_and_labels):
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
    
    return  evaluator.evaluate(predictions_and_labels)

def confusion_matrix_spark(predictions_and_labels):
    from pyspark.mllib.evaluation import MulticlassMetrics
    metricsp = MulticlassMetrics(predictions_and_labels.rdd)
    print(metricsp.confusionMatrix())
    return metricsp.confusionMatrix()

def classification_evaluation(predictions_and_labels, label_to_print):
    cm = confusion_matrix(predictions_and_labels,print_heat_cm=False,print_Pandas_cm=False, print_summary=False)
    print ("%s Accuracy: %3.3f" % (label_to_print, cm.ACC))
    print ("%s Error rate: %3.3f" % (label_to_print, 1-cm.ACC))
    print ("%s MCC: %3.3f" % (label_to_print, cm.MCC))
    return cm



In [2]:
#Inizialize Spark Context
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Brain_Data_Analysis").getOrCreate()
sc = spark.sparkContext

from pyspark.ml.linalg import SparseVector
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
import time
from pyspark.sql.functions import lit


#Training set Load
#The input dataset is composed of 4 files: Positive instances and negative instances (Training and Test)
#Each data row cointains a string of nucleotides: A G C T.
#We will load each line as a single experiment and each character as a feature. 

In [3]:
#Load Training Set

#Load Positive instances 
training_set=load_dna_dataset("/FileStore/tables/ipdata_tra_t_2018.txt",label_value=1.0)
#Read Negative instances
neg_tra=load_dna_dataset("/FileStore/tables/ipdata_tra_f_2018.txt",label_value=0.0)
#Join in a single Dataframe
training_set = training_set.union(neg_tra)

In [4]:
#A bit of exploration
training_set.head()
#training_set.printSchema()
#training_set.count()
#training_set.filter("label=1.0").count()
#training_set.filter("label=0.0").count()

Out[ 4 ]: Row(_1=SparseVector(4, {0: 1.0}), _2=SparseVector(4, {2: 1.0}), _3=SparseVector(4, {0: 1.0}), _4=SparseVector(4, {1: 1.0}), _5=SparseVector(4, {1: 1.0}), _6=SparseVector(4, {1: 1.0}), _7=SparseVector(4, {2: 1.0}), _8=SparseVector(4, {1: 1.0}), _9=SparseVector(4, {1: 1.0}), _10=SparseVector(4, {2: 1.0}), _11=SparseVector(4, {2: 1.0}), _12=SparseVector(4, {2: 1.0}), _13=SparseVector(4, {0: 1.0}), _14=SparseVector(4, {2: 1.0}), _15=SparseVector(4, {2: 1.0}), _16=SparseVector(4, {1: 1.0}), _17=SparseVector(4, {2: 1.0}), _18=SparseVector(4, {2: 1.0}), _19=SparseVector(4, {0: 1.0}), _20=SparseVector(4, {2: 1.0}), _21=SparseVector(4, {2: 1.0}), _22=SparseVector(4, {0: 1.0}), _23=SparseVector(4, {1: 1.0}), _24=SparseVector(4, {1: 1.0}), _25=SparseVector(4, {3: 1.0}), _26=SparseVector(4, {2: 1.0}), _27=SparseVector(4, {1: 1.0}), _28=SparseVector(4, {0: 1.0}), _29=SparseVector(4, {2: 1.0}), _30=SparseVector(4, {2: 1.0}), _31=SparseVector(4, {2: 1.0}), _32=SparseVector(4, {3: 1.0}), _33=SparseVector(4, {2: 1.0}), _34=SparseVector(4, {0: 1.0}), _35=SparseVector(4, {2: 1.0}), _36=SparseVector(4, {1: 1.0}), _37=SparseVector(4, {1: 1.0}), _38=SparseVector(4, {1: 1.0}), _39=SparseVector(4, {1: 1.0}), _40=SparseVector(4, {0: 1.0}), _41=SparseVector(4, {1: 1.0}), _42=SparseVector(4, {1: 1.0}), _43=SparseVector(4, {2: 1.0}), _44=SparseVector(4, {1: 1.0}), _45=SparseVector(4, {1: 1.0}), _46=SparseVector(4, {1: 1.0}), _47=SparseVector(4, {1: 1.0}), _48=SparseVector(4, {3: 1.0}), _49=SparseVector(4, {1: 1.0}), _50=SparseVector(4, {1: 1.0}), _51=SparseVector(4, {2: 1.0}), _52=SparseVector(4, {3: 1.0}), _53=SparseVector(4, {2: 1.0}), _54=SparseVector(4, {1: 1.0}), _55=SparseVector(4, {1: 1.0}), _56=SparseVector(4, {1: 1.0}), _57=SparseVector(4, {1: 1.0}), _58=SparseVector(4, {1: 1.0}), _59=SparseVector(4, {2: 1.0}), _60=SparseVector(4, {1: 1.0}), label=1.0)

In [5]:
#Load Test set
test_set=load_dna_dataset("/FileStore/tables/ipdata_test_t_2018.txt",label_value=1.0)
neg_test=load_dna_dataset("/FileStore/tables/ipdata_test_f_2018.txt",label_value=0.0)
test_set = test_set.union(neg_test)

In [6]:
#Spark-ML algorithms requires a single vector containing each features
#Assemble vector of features
assembler = VectorAssembler(inputCols=training_set.columns[0:len(training_set.columns)-1],outputCol="features")
training=assembler.transform(training_set).select("label","features")
#Note: we are unsing the same vector assembler instantiated for trainig set.
test=assembler.transform(test_set).select("label","features")

In [7]:
#A bit of exploration 
training.printSchema()
#training.head()
#training.show()

root
-- label: double (nullable = false)
-- features: vector (nullable = true)

In [8]:
#Cache data in memory.RDD (or Dataframe) are never persisted automatically by Spark.
training.cache()
test.cache()
print("Training Tot instances: %s" %training.count())
print("Test Tot instances: %s" %test.count())


Training Tot instances: 2002
Test Tot instances: 1186

In [9]:
#LOGISTIC REGRESSION
alg_label="LOGISTIC REGRESSION"
from pyspark.ml.classification import LogisticRegression
start_time = time.time()
lr_model = LogisticRegression()
lr_fitted = lr_model.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#LOGISTIC REGRESSION  TEST
start_time = time.time()
predictions_and_labels = lr_fitted.evaluate(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation
#cm=classification_evaluation(predictions_and_labels.predictions.select( "label", "prediction"),label_to_print=alg_label)
#Accuracy
predictions_and_labels.accuracy

LOGISTIC REGRESSION Training time: 3.23 seconds ---
LOGISTIC REGRESSION Test: 0.06 seconds ---
 Out[ 9 ]: 0.9477234401349073

In [10]:
#Decision TREE CLASSIFIER
alg_label="DECISION TREE CLASSIFIER"
from pyspark.ml.classification import DecisionTreeClassifier
start_time = time.time()
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",maxDepth=4)
dt_fitted = dt.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
dt_predictions_and_labels = dt_fitted.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
accuracy(dt_predictions_and_labels)

DECISION TREE CLASSIFIER Training time: 1.50 seconds ---
DECISION TREE CLASSIFIER Test: 0.13 seconds ---
 Out[ 10 ]: 0.9730185497470489

In [11]:
#Ramdom Forest Classifier TRAINING
alg_label="RANDOM FOREST CLASSIFIER"
from pyspark.ml.classification import RandomForestClassifier
start_time = time.time()
rf = RandomForestClassifier(labelCol="label", featuresCol="features",numTrees=100, maxDepth=15)
rf_fitted = rf.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
rf_predictions_and_labels = rf_fitted.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation    
#cm=classification_evaluation(rf_predictions_and_labels,label_to_print=alg_label)

RANDOM FOREST CLASSIFIER Training time: 13.47 seconds ---
RANDOM FOREST CLASSIFIER Test: 0.13 seconds ---

In [12]:
#Accuracy
rf_predictions_and_labels.filter("label=prediction").count()/rf_predictions_and_labels.count()
print(rf_fitted.toDebugString)

RandomForestClassificationModel (uid=RandomForestClassifier_08781c5a5261) with 100 trees
 Tree 0 (weight 1.0):
 If (feature 152 <= 0.5)
 If (feature 127 <= 0.5)
 If (feature 83 <= 0.5)
 Predict: 0.0
 Else (feature 83 > 0.5)
 If (feature 153 <= 0.5)
 Predict: 0.0
 Else (feature 153 > 0.5)
 If (feature 61 <= 0.5)
 If (feature 99 <= 0.5)
 Predict: 0.0
 Else (feature 99 > 0.5)
 If (feature 209 <= 0.5)
 If (feature 222 <= 0.5)
 Predict: 0.0
 Else (feature 222 > 0.5)
 If (feature 84 <= 0.5)
 Predict: 0.0
 Else (feature 84 > 0.5)
 Predict: 1.0
 Else (feature 209 > 0.5)
 If (feature 228 <= 0.5)
 Predict: 1.0
 Else (feature 228 > 0.5)
 Predict: 0.0
 Else (feature 61 > 0.5)
 Predict: 0.0
 Else (feature 127 > 0.5)
 If (feature 134 <= 0.5)
 If (feature 210 <= 0.5)
 If (feature 138 <= 0.5)
 If (feature 9 <= 0.5)
 If (feature 209 <= 0.5)
 If (feature 50 <= 0.5)
 If (feature 54 <= 0.5)
 Predict: 0.0
 Else (feature 54 > 0.5)
 If (feature 122 <= 0.5)
 Predict: 0.0
 Else (feature 122 > 0.5)
 If (feature 172 <= 0.5)
 Predict: 0.0
 Else (feature 172 > 0.5)
 Predict: 1.0
 Else (feature 50 > 0.5)
 If (feature 32 <= 0.5)
 Predict: 0.0
 Else (feature 32 > 0.5)
 If (feature 154 <= 0.5)
 Predict: 1.0
 Else (feature 154 > 0.5)
 Predict: 0.0
 Else (feature 209 > 0.5)
 If (feature 33 <= 0.5)
 If (feature 61 <= 0.5)
 Predict: 0.0
 Else (feature 61 > 0.5)
 If (feature 167 <= 0.5)
 Predict: 0.0
 Else (feature 167 > 0.5)
 If (feature 136 <= 0.5)
 Predict: 1.0
 Else (feature 136 > 0.5)
 Predict: 0.0
 Else (feature 33 > 0.5)
 If (feature 83 <= 0.5)
 Predict: 1.0
 Else (feature 83 > 0.5)
 Predict: 0.0
 Else (feature 9 > 0.5)
 If (feature 92 <= 0.5)
 If (feature 56 <= 0.5)
 If (feature 40 <= 0.5)
 If (feature 55 <= 0.5)
 Predict: 0.0
 Else (feature 55 > 0.5)
 If (feature 217 <= 0.5)
 Predict: 0.0
 Else (feature 217 > 0.5)
 Predict: 1.0
 Else (feature 40 > 0.5)
 If (feature 66 <= 0.5)
 Predict: 0.0
 Else (feature 66 > 0.5)
 Predict: 1.0
 Else (feature 56 > 0.5)
 If (feature 20 <= 0.5)
 Predict: 0.0
 Else (feature 20 > 0.5)
 Predict: 1.0
 Else (feature 92 > 0.5)
 If (feature 65 <= 0.5)
 If (feature 59 <= 0.5)
 Predict: 1.0
 Else (feature 59 > 0.5)
 Predict: 0.0
 Else (feature 65 > 0.5)
 Predict: 0.0
 Else (feature 138 > 0.5)
 If (feature 129 <= 0.5)
 If (feature 122 <= 0.5)
 Predict: 0.0
 Else (feature 122 > 0.5)
 If (feature 81 <= 0.5)
 Predict: 1.0
 Else (feature 81 > 0.5)
 If (feature 132 <= 0.5)
 If (feature 133 <= 0.5)
 If (feature 11 <= 0.5)
 Predict: 0.0
 Else (feature 11 > 0.5)
 Predict: 1.0
 Else (feature 133 > 0.5)
 Predict: 1.0
 Else (feature 132 > 0.5)
 Predict: 1.0
 Else (feature 129 > 0.5)
 Predict: 0.0
 Else (feature 210 > 0.5)
 If (feature 135 <= 0.5)
 If (feature 132 <= 0.5)
 If (feature 230 <= 0.5)
 If (feature 141 <= 0.5)
 If (feature 5 <= 0.5)
 If (feature 233 <= 0.5)
 If (feature 201 <= 0.5)
 Predict: 0.0
 Else (feature 201 > 0.5)
 Predict: 1.0
 Else (feature 233 > 0.5)
 Predict: 1.0
 Else (feature 5 > 0.5)
 Predict: 1.0
 Else (feature 141 > 0.5)
 Predict: 0.0
 Else (feature 230 > 0.5)
 If (feature 48 <= 0.5)
 Predict: 0.0
 Else (feature 48 > 0.5)
 Predict: 1.0
 Else (feature 132 > 0.5)
 If (feature 186 <= 0.5)
 If (feature 129 <= 0.5)
 If (feature 120 <= 0.5)
 If (feature 200 <= 0.5)
 If (feature 223 <= 0.5)
 Predict: 1.0
 Else (feature 223 > 0.5)
 If (feature 215 <= 0.5)
 Predict: 1.0
 Else (feature 215 > 0.5)
 Predict: 0.0
 Else (feature 200 > 0.5)
 If (feature 16 <= 0.5)
 If (feature 165 <= 0.5)
 If (feature 217 <= 0.5)
 Predict: 1.0
 Else (feature 217 > 0.5)
 Predict: 0.0
 Else (feature 165 > 0.5)
 Predict: 1.0
 Else (feature 16 > 0.5)
 If (feature 236 <= 0.5)
 Predict: 0.0
 Else (feature 236 > 0.5)
 Predict: 1.0
 Else (feature 120 > 0.5)
 Predict: 0.0
 Else (feature 129 > 0.5)
 Predict: 0.0
 Else (feature 186 > 0.5)
 Predict: 1.0
 Else (feature 135 > 0.5)
 If (feature 112 <= 0.5)
 Predict: 0.0
 Else (feature 112 > 0.5)
 If (feature 151 <= 0.5)
 If (feature 63 <= 0.5)
 Predict: 0.0
 Else (feature 63 > 0.5)
 If (feature 26 <= 0.5)
 Predict:

In [13]:
#Linear SVM TRAINING
alg_label="LINEAR SUPPORT VECTOR MACHINE"
from pyspark.ml.classification import LinearSVC
start_time = time.time()
lsvc = LinearSVC()
lsvc_fitted = lsvc.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
lsvc_predictions_and_labels = lsvc_fitted.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation    
#cm=classification_evaluation(lsvc_predictions_and_labels,label_to_print=alg_label)

LINEAR SUPPORT VECTOR MACHINE Training time: 20.06 seconds ---
LINEAR SUPPORT VECTOR MACHINE Test: 0.11 seconds ---

In [14]:
#Accuracy
lsvc_predictions_and_labels.filter("label=prediction").count()/lsvc_predictions_and_labels.count()

Out[ 14 ]: 0.9595278246205734

In [15]:
#Naive Bayes
alg_label="Naive Bayes"
from pyspark.ml.classification import NaiveBayes
start_time = time.time()
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
nb_fitted = nb.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
nb_predictions_and_labels = nb_fitted.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation    
nb_predictions_and_labels.filter("label=prediction").count()/nb_predictions_and_labels.count()

Naive Bayes Training time: 0.29 seconds ---
Naive Bayes Test: 0.02 seconds ---
 Out[ 22 ]: 0.9662731871838112

In [16]:
#Multilayer Perceptron Classifier
alg_label="Perceptron Classifier"

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
start_time = time.time()
# specify layers for the neural network:
# input layer of size 240 (Features)
# and output of size 2 (classes)
layers = [240, 40, 60, 2]
#layers = [240, 180, 50, 60, 2] 

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
# compute accuracy on the test set
start_time = time.time()
result = model.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy = " + str(evaluator.evaluate(predictionAndLabels)))
#cm=classification_evaluation(predictionAndLabels,label_to_print=alg_label)

Perceptron Classifier Training time: 17.39 seconds ---
Perceptron Classifier Test: 0.13 seconds ---
Accuracy = 0.9612141652613828

In [17]:
confusion_matrix_spark(predictionAndLabels)

DenseMatrix([[ 857., 27.],
 [ 19., 283.]])
 Out[ 24 ]: DenseMatrix(2, 2, [857.0, 19.0, 27.0, 283.0], 0)